In [ ]:
import os

import numpy as np
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 100)
pd.set_option('display.max_colwidth', 200)
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
%%html
<style>
.dataframe td {
    white-space: nowrap;
}
</style>

In [ ]:
path = os.path.abspath('data/acams2_data')


def load_data():
    _bo = pd.read_csv(path + '/BO.csv', low_memory=False)
    _cash = pd.read_csv(path + '/Cash.csv', low_memory=False)
    _customers = pd.read_csv(path + '/Customers.csv', low_memory=False)
    _ip = pd.read_csv(path + '/IP.csv', low_memory=False)
    _reg = pd.read_csv(path + '/Registration.csv', low_memory=False)
    _sdn = pd.read_csv(path + '/SDN.csv', low_memory=False)
    _wire = pd.read_csv(path + '/Wire.csv', low_memory=False)
    
    return _bo, _cash, _customers, _ip, _reg, _sdn, _wire

In [ ]:
bo, cash, cust, ip, reg, sdn, wire = load_data()

# Fix problems
cash = cash[['TRANSACTION_ID', 'TRANSACTION_DATE', 'DEBIT', 'CREDIT', 'CUSTOMER_ID', 'ACCOUNT_ID']]

cust['ACCOUNT_CREATION_DATE'] = pd.to_datetime(cust['ACCOUNT_CREATION_DATE'], format='%m/%d/%Y')
cust['ACCOUNT_UPDATE_DATE'] = pd.to_datetime(cust['ACCOUNT_UPDATE_DATE'], format='%m/%d/%Y')
cust.sort_values(by=['ACCOUNT_CREATION_DATE', 'ACCOUNT_UPDATE_DATE'], inplace=True)

wire['TRANSACTION_DATE'] = pd.to_datetime(wire['TRANSACTION_DATE'], format='%m/%d/%Y')

reg['REGISTRATION_DATE'] = pd.to_datetime(reg['REGISTRATION_DATE'], format='%m/%d/%Y')

cash['TRANSACTION_DATE'] = cash['TRANSACTION_DATE'].apply(lambda x: x if x != '1/16/21' else '1/16/2021')  # TATO TRANSAKCE JE DIVNA, JAKO JEDINA MA JINY FORMAT DATA
cash['TRANSACTION_DATE'] = pd.to_datetime(cash['TRANSACTION_DATE'], format='%m/%d/%Y')

# Do not change the following lines unless you perfectly know what you are doing!!!
bo.replace(to_replace=[np.nan], value=None, inplace=True)
reg.replace(to_replace=[np.nan], value=None, inplace=True)
ip.replace(to_replace=[np.nan], value=None, inplace=True)
cust.replace(to_replace=[np.nan], value=None, inplace=True)
wire.replace(to_replace=[np.nan], value=None, inplace=True)
cash.replace(to_replace=[' '], value=None, inplace=True)

In [ ]:
# DO NOT CHANGE UNLESS YOU ARE SURE WHAT YOU ARE DOING!!!

# Process customer table
alpha = cust.groupby(['CUSTOMER_ID', 'ACCOUNT_ID', 'ACCOUNT_TYPE']).filter(lambda g: len(g) > 1)
alpha_first = alpha.drop_duplicates(subset=['CUSTOMER_ID', 'ACCOUNT_ID', 'ACCOUNT_TYPE'], keep='first')  # Updated account info

alpha_last = alpha.drop_duplicates(subset=['CUSTOMER_ID', 'ACCOUNT_ID', 'ACCOUNT_TYPE'], keep='last').copy()  # Old address, phone etc.
alpha_last.rename(columns={'STREET': 'OLD_STREET', 'CITY': 'OLD_CITY', 'STATE': 'OLD_STATE', 'POSTAL_CODE': 'OLD_POSTAL_CODE', 'COUNTRY': 'OLD_COUNTRY', 'COUNTRY_CODE': 'OLD_COUNTRY_CODE', 'PHONE': 'OLD_PHONE'}, inplace=True)
alpha_last = alpha_last[['ACCOUNT_ID', 'OLD_STREET', 'OLD_CITY', 'OLD_STATE', 'OLD_POSTAL_CODE', 'OLD_COUNTRY', 'OLD_COUNTRY_CODE', 'OLD_PHONE']]

beta = cust.groupby(['CUSTOMER_ID', 'ACCOUNT_ID', 'ACCOUNT_TYPE']).filter(lambda g: len(g) == 1)

beta_first = beta.sort_values(by=['ACCOUNT_TYPE']).drop_duplicates(subset=['CUSTOMER_ID', 'ACCOUNT_ID'], keep='first')
only_savings_account_ids = beta_first[beta_first['ACCOUNT_TYPE'] == 'Savings']['ACCOUNT_ID'].to_list()

beta_last = beta.sort_values(by=['ACCOUNT_TYPE']).drop_duplicates(subset=['CUSTOMER_ID', 'ACCOUNT_ID'], keep='last')
also_savings_account_ids = beta_last[(beta_last['ACCOUNT_TYPE'] == 'Savings') & (~beta_last['ACCOUNT_ID'].isin(only_savings_account_ids))]['ACCOUNT_ID'].to_list()

new_cust = pd.concat(objs=[beta_last, alpha_first], axis=0)
new_cust['ACCOUNT_STRUCTURE'] = new_cust['ACCOUNT_ID'].apply(lambda x: 'Savings' if x in only_savings_account_ids else ('Both' if x in also_savings_account_ids else 'Checking'))
new_cust = pd.merge(new_cust, alpha_last, how='left', on='ACCOUNT_ID').replace(to_replace=[np.nan], value=None)

# Merge business ownership and company registration tables
bo_reg = pd.merge(bo, reg, how='outer', on='COMPANY_ID', validate='many_to_one').replace(to_replace=[np.nan], value=None)
bo_reg['BENEFICIARY_ID'] = bo_reg['BENEFICIARY_ID'].astype(float)

# Merge wire with new customer table
__df =  pd.merge(wire, new_cust, how='left', on='ACCOUNT_ID', suffixes=('_wire', '_cust')).replace(to_replace=[np.nan], value=None)

# Merge wire with merged business own. and company reg. table
_df = pd.merge(__df, bo_reg, how='left', on='BENEFICIARY_ID', suffixes=('_wire', '_bene')).replace(to_replace=[np.nan], value=None)

# Merge wire with ip table
df_wire = pd.merge(_df, ip, how='left', on='IP_ADDRESS', suffixes=('_wire', '_ip')).replace(to_replace=[np.nan], value=None)

# Add transaction type
df_wire['TRANSACTION_TYPE'] = 'Wire'

In [ ]:
df_wire

In [ ]:
df_wire.to_csv('data/wire.csv')

In [ ]:
# Cash table
_df = pd.merge(cash, new_cust, how='left', on='ACCOUNT_ID', suffixes=('_cash', '_cust')).replace(to_replace=[np.nan], value=None)
df_cash = pd.merge(_df, bo_reg, how='left', left_on='CUSTOMER_ID_cash', right_on='BENEFICIARY_ID', suffixes=('_cash', '_bene')).replace(to_replace=[np.nan], value=None)
df_cash['TRANSACTION_TYPE'] = 'Cash'

In [ ]:
df_cash.to_csv('data/cash.csv')

In [ ]:
# Interesting

df_wire[df_wire['CUSTOMER_ID_wire'] != df_wire['CUSTOMER_ID_cust']]

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../data/wire.csv")